In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Basics of Neural Networks

# Pytorch Tutorial

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

## Tensor Operations

In [ ]:
# Pytorch functions similarly to NumPy and shares many common operations (sometimes with different syntax ..)
x = torch.rand(5, 3)
y = torch.rand(5, 3)
z = x * y
print(x, y, z)
print(x.shape, y.shape, z.shape)

x = x.reshape(1, -1)
y = y.reshape(-1, 1)
z = x @ y
print(x, y, z)
print(x.shape, y.shape, z.shape)
print(z.item())

In [ ]:
# Pytorch tensors can be converted to NumPy arrays and vice-versa
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

c = torch.tensor(b)
print(c)

In [ ]:
# Tensors can be moved between CPU and GPU using the .to method
if torch.cuda.is_available():
  device = torch.device('cuda')
  x = torch.rand(12, 12).to(device)
  evals, evecs = torch.eig(x, eigenvectors=True)
  print(evals.cpu().numpy())
  print(evecs.cpu().numpy())

## Pytorch Modules

In [ ]:
# Most NN operations and computational graphs are represented as `torch.nn.Module` objects
# For a summary of all NN building blocks, see https://pytorch.org/docs/stable/nn.html
fc = nn.Linear(in_features=12, out_features=1)
print(fc)

X0 = torch.rand(5, 12)
X1 = fc(X0)
print(X1)
print(X0.shape, X1.shape)

print(fc.weight, fc.bias)

## Calculating Gradients via Backpropogation





In [ ]:
# Pytorch will construct a computational graph for backpropagation for variables that require gradients
# and variables that depend on them
x = torch.tensor([2.]).requires_grad_(True)
y = x**2

# The backward pass can be initiated by calling the backward function on a variable
y.backward()
print(x, y)

# The gradient is stored as a .grad attribute after calling the backward function
print(x.grad)

In [ ]:
# The gradients of vectors and matrices can be done with a few tricks
x = torch.linspace(-4, 4, 50).cuda().requires_grad_(True)
y = x**2
print(x, y)
# The backward pass can be initiated by calling the backward function on a variable
y.backward(torch.ones(50,).cuda())
print(x.grad)

x_grad_n = x.grad.detach().cpu().numpy()
x_n = x.detach().cpu().numpy()
y_n = y.detach().cpu().numpy()

plt.plot(x_n, y_n)
plt.plot(x_n, x_grad_n)
plt.xlabel('x')
plt.ylabel('y and grad_y')
plt.legend(['y', 'grad_y'])

## Question: Calculate the gradient of sin(x) on the interval [0, 2pi]

# Building Neural Networks with `torch.nn.Sequential`

In [ ]:
## The torch.nn.Sequential module can be used to construct sequential computation graphs
model = torch.nn.Sequential(nn.Linear(12, 12),
                            nn.ReLU(),
                            nn.Linear(12, 1))
X = torch.rand(5, 12)
out = model(X)
print(out.shape)

# Example: Learning the XOR Problem

## Construct dataset

In [ ]:
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]]).float().to(device)
y = torch.tensor([0, 1, 1, 0]).long().to(device)

## Construct model `torch.nn.Sequential`

In [ ]:
model = torch.nn.Sequential(nn.Linear(2, 10),
                            nn.ReLU(),
                            nn.Linear(10, 2))
model = model.to(device)

## Construct optimizer `torch.optim.SGD`

In [ ]:
# The optimizer requires you pass a list of all adjustable parameters as well as the learning rate
optimizer = optim.SGD(model.parameters(), lr=1.0)
ce_loss = nn.CrossEntropyLoss()

## Perform stochastic gradient descent

In [ ]:
for epoch in range(100):
  model.zero_grad()
  out = model(X)
  loss = ce_loss(out, y)
  loss.backward()
  optimizer.step()
  if epoch%10 == 0: print("Epoch: {} Loss: {}".format(epoch, loss))

## Evaluate model

In [ ]:
from sklearn.metrics import accuracy_score
preds_scores = model(X)
print(preds_scores)
print(nn.functional.softmax(preds_scores, dim=1))
preds = nn.functional.softmax(preds_scores, dim=1).max(dim=1)[1].detach().cpu().numpy()
targets = y.cpu().numpy()
print(accuracy_score(targets, preds))

## Question: What is the smallest neural network that can learn the XOR function?

# Example: Identifying Magnetic Materials with Chemical Formulas

## Download dataset

In [ ]:
!pip install matminer
!pip install skorch

In [ ]:
from matminer.datasets.convenience_loaders import load_mp
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty

In [ ]:
df = load_mp()  # loads dataset in a pandas DataFrame object
df.head()

In [ ]:
df.describe()

## Featurize chemical formula dataset

In [ ]:
unwanted_columns = ['mpid', 'e_hull', 'gap pbe', 'elastic anisotropy',
       'bulk modulus', 'shear modulus', 'e_form']
df = df.drop(unwanted_columns, axis=1)
df.head()

In [ ]:
# Convert formula to composition
df = StrToComposition().featurize_dataframe(df, "formula", ignore_errors=True) 

# Create features based on composition
ep_feat = ElementProperty.from_preset(preset_name="magpie") 

# input the "composition" column to the featurizer
df = ep_feat.featurize_dataframe(df, col_id="composition", ignore_errors=True)  

# drop rows with NaN values
df = df.dropna(axis=0) 
df.head()

In [ ]:
# Convert mu_b to magnetization classification problem
y = df['mu_b'].values
y = np.abs(y) > 1e-6

# Drop non-numerical features and tasks from data frame
excluded = ['mu_b', 'formula', 'composition']
X = df.drop(excluded, axis=1).values

# Standardize input data
X = (X-X.mean(axis=-1, keepdims=True))/X.std(axis=-1, keepdims=True)

# Convert numpy array to pytorch
X = torch.tensor(X).float()
y = torch.tensor(y).float()
print(X.shape, y.shape)

## Construct model

In [ ]:
# Construct model (try changing dim!)
num_features = X.shape[-1]
dim = 1024

model = nn.Sequential(nn.Linear(num_features, dim),
                      nn.ReLU(),
                      nn.Linear(dim, dim),
                      nn.ReLU(),
                      nn.Linear(dim, 1))

## Use `skorch` to handle optimization

In [ ]:
from skorch import NeuralNetBinaryClassifier
net = NeuralNetBinaryClassifier(model, max_epochs=20, lr=1e-6, device='cuda')
net.fit(X, y)